In [13]:
#imports and basic splits
import numpy as np
import tensorflow.keras as kr
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
data = np.load("EEG_useful.npy", allow_pickle = True)
X_data = data[:, :, 2:11]
Y_data = data[:, :, 11]
X_data = X_data.astype(int)
Y_data = Y_data.astype(int)
Y_data = Y_data[:, 0:110:110]

In [14]:
#data split
x_train, x_test, y_train, y_test = train_test_split(X_data, Y_data, train_size=0.65,test_size=0.35, random_state=101)
print(x_train.shape)

(65, 110, 9)


In [15]:

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)
print(y_test.shape)


(35, 2)


In [16]:
#wydzielenie wykresow i normalizacja | singling out graphs and normalization
x_train_arr = []
x_test_arr = []
for i in range(x_train.shape[2]): 
    x_train_arr.append(x_train[:, :, i])
    x_test_arr.append(x_test[:, :, i])
    #x_train_arr[i] = x_train_arr[i].reshape(x_train_arr[i].shape[0], x_train_arr[i].shape[1], 1)
    #x_test_arr[i] = x_test_arr[i].reshape(x_test_arr[i].shape[0], x_test_arr[i].shape[1], 1)
    max_value = np.max(x_train_arr[i])
    if max_value > np.max(x_test_arr[i]):
        max_value = np.max(x_test_arr[i])
    x_train_arr[i], x_test_arr[i] = x_train_arr[i].astype(float)/(max_value/10), x_test_arr[i].astype(float)/(max_value/10)


In [24]:
#keras model using functional API, every graph type is its own input, three sequential layers, later concatenated
input_arr = []
branch_arr = []
for i in range(x_train.shape[2]):
    input_arr.append(kr.Input(shape = (x_train_arr[i].shape[1])))
    branch_arr.append(None)
    branch_arr[i] = Dense(64, activation = "relu")(input_arr[i])
    branch_arr[i] = Dense(32, activation = "relu")(branch_arr[i])
    branch_arr[i] = Dropout(0.10)(branch_arr[i])
    branch_arr[i] = Dense(16, activation = "relu")(branch_arr[i])
    branch_arr[i] = kr.Model(inputs = input_arr[i], outputs = branch_arr[i])
combined_branch = kr.layers.concatenate([branch_arr[i].output for i in range(x_train.shape[2])])
combined_NN = Dense(32, activation = "relu")(combined_branch)
combined_NN = Dense(2, activation="softmax")(combined_NN)
EEG_model_MI = kr.Model(inputs = input_arr, outputs = combined_NN)

In [25]:
EEG_model_MI.compile(loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['accuracy'])
batch_size = 2
n_epochs = 10
EEG_model_MI.summary()


Model: "model_49"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           [(None, 110)]        0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           [(None, 110)]        0                                            
__________________________________________________________________________________________________
input_39 (InputLayer)           [(None, 110)]        0                                            
__________________________________________________________________________________________________
input_40 (InputLayer)           [(None, 110)]        0                                            
___________________________________________________________________________________________

In [26]:
#print([x_train_arr[i].shape for i in range(len(x_train_arr))])
EEG_model_MI.fit([x_train_arr[i] for i in range(len(x_train_arr))], y_train, batch_size=batch_size,
          epochs=n_epochs,
          verbose=1,
          validation_data=([x_test_arr[i] for i in range(len(x_test_arr))], y_test))
EEG_model_MI.save("EEG_model_MI")

Epoch 1/10
33/33 [==============================] - 1s 20ms/step - loss: 0.7570 - accuracy: 0.5538 - val_loss: 0.6376 - val_accuracy: 0.7143
Epoch 2/10
33/33 [==============================] - 0s 7ms/step - loss: 0.4574 - accuracy: 0.7692 - val_loss: 0.6523 - val_accuracy: 0.6286
Epoch 3/10
33/33 [==============================] - 0s 8ms/step - loss: 0.2801 - accuracy: 0.9538 - val_loss: 0.5812 - val_accuracy: 0.7143
Epoch 4/10
33/33 [==============================] - 0s 8ms/step - loss: 0.1882 - accuracy: 0.9692 - val_loss: 0.5762 - val_accuracy: 0.6286
Epoch 5/10
33/33 [==============================] - 0s 7ms/step - loss: 0.1078 - accuracy: 1.0000 - val_loss: 0.6323 - val_accuracy: 0.6857
Epoch 6/10
33/33 [==============================] - 0s 8ms/step - loss: 0.0841 - accuracy: 0.9692 - val_loss: 0.5352 - val_accuracy: 0.6857
Epoch 7/10
33/33 [==============================] - 0s 8ms/step - loss: 0.0618 - accuracy: 1.0000 - val_loss: 0.6804 - val_accuracy: 0.5714
Epoch 8/10
33/33 [=

In [27]:
EEG_model_MI.evaluate([x_test_arr[i] for i in range(len(x_test_arr))], y_test, batch_size = 2)

18/18 [==============================] - 0s 2ms/step - loss: 0.6723 - accuracy: 0.6857


[0.6722670793533325, 0.6857143044471741]